In [1]:
!pip install albumentations==1.0.3
!pip install torch torchvision
!pip install pyyaml
!pip install opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 2.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determ

In [2]:
!git clone https://github.com/ultralytics/yolov3
%cd yolov3
!pip install -r requirements.txt


fatal: destination path 'yolov3' already exists and is not an empty directory.
/content/yolov3


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import os
import yaml

In [41]:
!pip install ultralytics

In [4]:
import matplotlib.pyplot as plt

# Add the YOLOv3 repository to the Python path
import sys
sys.path.append('/content/yolov3/models')

# Import YOLOv3 specific functions and classes
from ultralytics import YOLO
model = YOLO("yolov3-tiny.pt")


PRO TIP 💡 Replace 'model=yolov3-tiny.pt' with new 'model=yolov3-tinyu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 23.3M/23.3M [00:00<00:00, 194MB/s]


In [4]:
preprocess = A.Compose([
    A.SmallestMaxSize(max_size=640),
    A.Resize(height=640, width=640),
    A.Equalize(always_apply=True),
    A.GridDistortion(num_steps=2, distort_limit=0.3, p=1.0),
])

In [5]:
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.19, contrast_limit=0.19, p=0.5),
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
        A.MotionBlur(blur_limit=3, p=0.5),
    ], p=0.5),
    A.OneOf([
        A.ToGray(p=0.15),
    ], p=1.0),
    ToTensorV2(),
])

In [6]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, split, preprocess, augment=None):
        self.root_dir = root_dir
        self.split = split
        self.preprocess = preprocess
        self.augment = augment

        self.image_dir = os.path.join(self.root_dir, self.split, 'images')
        self.label_dir = os.path.join(self.root_dir, self.split, 'labels')
        self.classes_file = os.path.join(self.root_dir, self.split, '_classes.txt')

        self.image_list = os.listdir(self.image_dir)
        self.classes = self._load_classes()

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = self.image_list[idx]
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, img_name.replace('.jpg', '_annotations.txt'))

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read labels
        boxes = []
        class_labels = []
        with open(label_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                class_label, x_center, y_center, width, height = map(float, line.strip().split())
                class_labels.append(int(class_label))
                boxes.append([x_center, y_center, width, height])

        boxes = np.array(boxes)
        class_labels = np.array(class_labels)

        transformed = self.preprocess(image=image, bboxes=boxes, class_labels=class_labels)
        image = transformed['image']
        boxes = transformed['bboxes']
        class_labels = transformed['class_labels']

        if self.augment:
            augmented = self.augment(image=image, bboxes=boxes)
            image = augmented['image']
            boxes = augmented['bboxes']

        return image, {'boxes': boxes, 'labels': class_labels}

    def _load_classes(self):
        with open(self.classes_file, 'r') as f:
            classes = [line.strip() for line in f.readlines()]
        return classes

In [7]:
# Load dataset
root_dir = "/content/drive/MyDrive/"
train_dataset = CustomDataset(root_dir=root_dir, split='goldfish_dataset', preprocess=preprocess, augment=augment)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [8]:
def train(model, dataloader, epochs, device):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        num_batches = 0

        for images, targets in dataloader:
            images = images.to(device)
            boxes = targets['boxes'].to(device)
            labels = targets['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            num_batches += 1

        avg_loss = epoch_loss / num_batches
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}")

In [ ]:
# Initialize and train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ROOT_DIR = '/content/drive/MyDrive/Colab Notebooks'
results = model.train(data=os.path.join(ROOT_DIR, "google_colab_config.yaml"), epochs=10)
#train(model, train_dataloader, epochs=10, device=device)

Ultralytics YOLOv8.2.27 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov3-tiny.pt, data=/content/drive/MyDrive/Colab Notebooks/google_colab_config.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, s

train: Scanning /content/drive/MyDrive/goldfish_dataset/labels/train... 0 images, 9610 backgrounds, 0 corrupt: 100%|██████████| 9610/9610 [03:52<00:00, 41.39it/s] 

train: WARNING ⚠️ No labels found in /content/drive/MyDrive/goldfish_dataset/labels/train.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


train: New cache created: /content/drive/MyDrive/goldfish_dataset/labels/train.cache
WARNING ⚠️ No labels found in /content/drive/MyDrive/goldfish_dataset/labels/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/goldfish_dataset/labels/val... 0 images, 888 backgrounds, 0 corrupt: 100%|██████████| 888/888 [00:13<00:00, 67.08it/s] 

val: WARNING ⚠️ No labels found in /content/drive/MyDrive/goldfish_dataset/labels/val.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


val: New cache created: /content/drive/MyDrive/goldfish_dataset/labels/val.cache
WARNING ⚠️ No labels found in /content/drive/MyDrive/goldfish_dataset/labels/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs/detect/train/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 19 weight(decay=0.0), 24 weight(decay=0.0005), 23 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, cli

       1/10         0G          0      249.8          0          0        640:   2%|▏         | 11/601 [05:28<4:39:04, 28.38s/it]